In [31]:
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
#from pandas import Series,DataFrame

pd.set_option('display.max_rows', 500) 
filepath = '/Users/Suad/FB-data/'
#frame = pd.read_csv(filepath+'Syriapages.csv')
frame = pd.read_json(filepath+'Syria-FB-pages.json')
print "loaded Records "
print frame.name.count()

slicee = DataFrame(frame)
#[['name','category','talking_about_count','engagement','likes','start_info_clean']]
#print slicee
main_cat = ['Society/Culture Website', 'Travel/Leisure' ,'Transport/Freight', 'Work Position', 'Community', 'Small Business'
            , 'Society/Culture Website', 'Business Person', 'Business Services','Cause', 'Community Organization', 
            'Community/Government' , 'Company' , 'Consulting/Business Services', 'Regional Website', 'Media/News/Publishing'
             ,'Local/Travel Website', 'Local Business']
important_pages= DataFrame()
#columns=['name','category','talking_about_count','engagement','likes','start_info_clean']
#print important_pages.columns
for cat in main_cat:
    #print cat
    #print "--------"
    temp = slicee[slicee.category == cat]
    #print temp.name.count()
    #print temp.name
    important_pages = important_pages.append(temp) 

print "Records count with important categories:"
print important_pages.name.count()
important_pages = important_pages[~important_pages.name.str.contains(u'عشاق|دوري|كرة|عشق|عاشق|الله|أبطال|فتيات|اسلام|ابطال|بنات')]
print "Records count after cleaning keywords:"
print important_pages.name.count()
important_pages = important_pages[important_pages.likes!=0]
print "Records count after removing zero-engagment page:"
print important_pages.name.count()


# sort 
important_pages = important_pages.sort_values(by=['start_info_clean','talking_about_count','likes'], ascending=[1,0,0]) 
idlist=[]
for i in important_pages.id:
    if(i>0):
        idlist.append(int(i))
f = open(filepath+"filtered_syrian_pages_ids_only.txt", "w")
f.write("\n".join(map(lambda x: str(x), idlist())))

#important_pages.reset_index().to_json(filepath+"filtered_syrian_pages.json",orient='records')

loaded Records 
1149
Records count with important categories:
486
Records count after cleaning keywords:
396
Records count after removing zero-engagment page:
359


TypeError: 'list' object is not callable

In [49]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from pymongo import MongoClient
import logging,re,time
import requests
#import textblob
#import langid
import collections
import numpy as np

pd.set_option('display.max_rows', 100) 
filepath = '/Users/Suad/FB-data/'
client = MongoClient()
db = client.unicief


logging.basicConfig(level=logging.INFO,
                    filename='log.log', # log to this file
                    format='%(asctime)s %(message)s') # include timestamp

pagesCollection = db.syrpages
postsCollection = db.syrposts
commentsCollection = db.comments
likesCollection=db.syrlikes


print "loaded Records "
print pagesCollection.count()
#Set All False
pagesCollection.update_many({},{'$set':{'relevant':False}})
print "Set all false"
print pagesCollection.count()

slicee = DataFrame(list(pagesCollection.find()))

#[['name','category','talking_about_count','engagement','likes','start_info_clean']]
#print slicee
main_cat = ['Society/Culture Website', 'Travel/Leisure' ,'Transport/Freight', 'Work Position', 'Community', 
            'Small Business',
            'Society/Culture Website', 'Business Person', 'Business Services','Cause', 'Community Organization', 
            'Community/Government' , 'Company' , 'Consulting/Business Services', 'Regional Website', 'Media/News/Publishing'
             ,'Local/Travel Website', 'Local Business']

#false_kw =[u'عشاق','دوري','كرة','عشق','عاشق','الله','أبطال','فتيات','اسلام','ابطال','بنات'];
false_kw =u'عشاق|دوري|كرة|عشق|عاشق|الله|أبطال|فتيات|اسلام|ابطال|بنات'
#Loop pages
for pid in pagesCollection.distinct("id"):
    pagesCollection.update_one({'id':pid , 'category': { '$in': main_cat } },{'$set':{'relevant':True}})

print "Total pages after set relevant categories"
relevant= pagesCollection.find({'relevant':True})
print relevant.count()

import re
regexp = re.compile(false_kw)
for page in relevant:    
    if (regexp.search(page['name']) is not None):
        pagesCollection.update_one({'id':page['id']} ,{'$set':{'relevant':False}})
        #log 
        #print "removed:" + page['name']
    if (page['likes']==0):
        pagesCollection.update_one({'id':page['id']} ,{'$set':{'relevant':False}})    
        #log 
        # print "removed:" + page['name']
#for pid in relevant.distinct("id"):

print "Total pages after remove false kw and zero engagment"
relevant= pagesCollection.find({'relevant':True})
print relevant.count()

loaded Records 
1151
Set all false
1151
Total pages after set relevant categories
479
Total pages after remove false kw and zero engagment
352


In [37]:
#loop relevant 
limit=0
for page in relevant: 
    limit +=1
    if(limit>5):
        break
    print page['name']
    print page['id']
    #g.add_vertices(page["id"])
    # get page posts 
    posts = postsCollection.find({"page_id":page['id']})
    limitp =0
    for post in posts:
        limitp+=1
        if(limitp >5):
            break
        
        if "story_tags" in post:
            print post['story_tags']
            
        
        #g.add_vertices(post["id"])
    # get users

الهجرة الى اوروبا من تركيا
378927898868694
طرق اللجوء الى اوروبا
152570638271216
{u'1': [{u'length': 21, u'type': u'page', u'id': u'152570638271216', u'name': u'\u0637\u0631\u0642 \u0627\u0644\u0644\u062c\u0648\u0621 \u0627\u0644\u0649 \u0627\u0648\u0631\u0648\u0628\u0627', u'offset': 1}]}
أوروبا
776373632471822
{u'1': [{u'length': 6, u'type': u'page', u'id': u'776373632471822', u'name': u'\u0623\u0648\u0631\u0648\u0628\u0627', u'offset': 1}]}
{u'1': [{u'length': 6, u'type': u'page', u'id': u'776373632471822', u'name': u'\u0623\u0648\u0631\u0648\u0628\u0627', u'offset': 1}]}
اخبار المهاجرين في اوروبا
307091372652912
{u'1': [{u'length': 25, u'type': u'page', u'id': u'307091372652912', u'name': u'\u0627\u062e\u0628\u0627\u0631 \u0627\u0644\u0645\u0647\u0627\u062c\u0631\u064a\u0646 \u0641\u064a \u0627\u0648\u0631\u0648\u0628\u0627', u'offset': 1}]}
اوروبا مستقبلي
449543831789061
{u'1': [{u'length': 14, u'type': u'page', u'id': u'449543831789061', u'name': u'\u0627\u0648\u0631\u0648\u0628\

In [253]:
from networkx.algorithms import bipartite
limit =0
result =[]
relevant= pagesCollection.find({'relevant':True},{"id":1, "_id":0})

#relevant.toArrays(function(u){ result.push(u.text) })
x = 449543831789061
print result
B = nx.Graph()
usersGraph = nx.Graph()
relevant_pages=[]
posts =[]
net={}
for x in relevant:
    relevant_pages.append(x["id"]) # find better way todo this
    
print likesCollection.count()
print commentsCollection.count()
print postsCollection.count()
for comment in commentsCollection.find().limit(10):
    
    ids = comment['id'].split('_')
    key = post_id = ids[0]
    #p= postsCollection.find({'id':key},{"page_id":1, "_id":0})
    #print list(p)
    #post_id =ids[1]
    user = comment['from']['id']
    if key not in net:
        net[key] = [user]
    else:
        #print "found"
        if user not in net[key]:
            net[key].append(user)

B.add_nodes_from(net.keys(),Bipartite=0)
        
for post in net:
        B.add_nodes_from(list(net[post]),Bipartite=1)
        for value in net[post]:
            B.add_edge(post,value)
            
users_nodes, posts_nodes = bipartite.sets(B)
print "posts:",posts_nodes
print "users:",users_nodes
nx.draw(G)

[]
3590460
317666
124232
posts: set([u'916807138406205'])
users: set([u'140724502942173', u'724151681050049', u'712707625529964', u'701472406650308', u'727409774060908', u'1641450669473438', u'1521603274798444', u'511741552335864', u'506190092877137', u'140119983003128'])


In [174]:
import networkx as nx
B = nx.Graph()
B.add_nodes_from([1,2,3,4], bipartite=0) # Add the node attribute "bipartite"
B.add_nodes_from(['a','b','c'], bipartite=1)
B.add_edges_from([(1,'a'), (1,'b'), (2,'b'), (2,'c'), (3,'c'), (4,'a')])
print B.edges()
print B.nodes()

[('a', 1), ('a', 4), (1, 'b'), (2, 'c'), (2, 'b'), (3, 'c')]
['a', 1, 2, 3, 4, 'c', 'b']
